In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, GRU, Flatten
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.linear_model import LogisticRegression
from keras.layers import CuDNNLSTM, Dropout, Dense,Conv1D, MaxPooling1D


#import GPFramework.data as data

Using TensorFlow backend.


In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

W1122 17:50:37.741700 4340395456 deprecation_wrapper.py:119] From /Users/brandonsu/anaconda3/lib/python3.7/site-packages/Keras-2.2.4-py3.7.egg/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1122 17:50:37.745125 4340395456 deprecation_wrapper.py:119] From /Users/brandonsu/anaconda3/lib/python3.7/site-packages/Keras-2.2.4-py3.7.egg/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1122 17:50:37.748610 4340395456 deprecation_wrapper.py:119] From /Users/brandonsu/anaconda3/lib/python3.7/site-packages/Keras-2.2.4-py3.7.egg/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1122 17:50:37.765324 4340395456 deprecation_wrapper.py:119] From /Users/brandonsu/anaconda3/lib/python3.7/site-packages/Keras-2.2.4-py3.7.egg/keras/backend/tensorflow_backend.py:190: T

[]

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [4]:
train

,"Absolutely enjoyable singing and dancing movie starring Frank Sinatra and gen Kelly, as well as Kathryn Grayson.<br /><br />The film won and Oscar for George E. Stoll's score, and it garnered nominations for Best Picture, Best Actor for Kelly, and Best Cinematography, as well as a Best Son nomination for ""I Fall in Love Too Easily"" sung by Sinatra.<br /><br />It was a cute story about Kelly helping his pal Sinatra get a girl and falling in love with her himself. The lovely Grayson (The Toast of New Orleans) dazzled us with her singing, and we had a lot of great songs and dance routines by Kelly and Sinatra, as well as the artistry of pianist-conductor José Iturbi.<br /><br />A classic Hollywood music from an era gone by.",1
0,After all these years I still consider this se...,1
1,A hilarious and insightful perspective of the ...,1
2,After reading all of the rave reviews about th...,0
3,"""Tintin and I"" first of all struck me as a mas...",1
4,The latest film by the Spanish director Agusti...,1
5,"Okay, I absolutely LOVE Ben Stiller, although ...",0
6,"Okay, like many other such films, spawned out ...",1
7,If the very thought of Arthur Askey twists you...,1
8,Jean-Claude Van Damme plays twin brothers Alex...,0
9,"""The Invisible Mouse"" is a delightful and diff...",1


In [5]:
# Dataset is well balanced
print(train[train.iloc[:,1] == 0].size)
print(train[train.iloc[:,1] == 1].size)

print(test[test.iloc[:,1] == 0].size)
print(test[test.iloc[:,1] == 1].size)

25000
24998
25000
24998


In [6]:
x_train = train.iloc[:,0]
y_train = train.iloc[:,1]

x_test = test.iloc[:,0]
y_test = test.iloc[:,1]

In [7]:
tokenizer = Tokenizer(num_words=4000)
tokenizer.fit_on_texts(x_train)
X_train = tokenizer.texts_to_sequences(x_train)
X_test = tokenizer.texts_to_sequences(x_test)


In [8]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 400
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
vocab_size = np.amax(X_train) + 1

In [9]:
# classifier = LogisticRegression()
# classifier.fit(X_train, y_train)
# score = classifier.score(X_test, y_test)

# print("Accuracy:", score)

In [10]:
input_dim = X_train.shape[1]

In [11]:
model = Sequential()
# embedding_layer = Embedding(vocab_size, 1500, input_length=maxlen , trainable=False)
# model.add(embedding_layer)
# model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=400))
# model.add(Dropout(0.2))
# model.add(Conv1D(64, 5, activation='tanh'))
# model.add(MaxPooling1D(pool_size=4))
# model.add(CuDNNLSTM(128))
model.add(LSTM(16))
#model.add(Dense(32, activation='relu'))
#model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

W1122 17:50:51.944122 4340395456 deprecation_wrapper.py:119] From /Users/brandonsu/anaconda3/lib/python3.7/site-packages/Keras-2.2.4-py3.7.egg/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1122 17:50:52.132801 4340395456 deprecation_wrapper.py:119] From /Users/brandonsu/anaconda3/lib/python3.7/site-packages/Keras-2.2.4-py3.7.egg/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1122 17:50:52.153244 4340395456 deprecation.py:323] From /Users/brandonsu/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 128)          512000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 16)                9280      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 521,297
Trainable params: 521,297
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train,epochs=10)

Epoch 1/10
24999/24999 [==============================] - 190s 8ms/step - loss: 0.6890 - acc: 0.5176
Epoch 2/10
24999/24999 [==============================] - 201s 8ms/step - loss: 0.6652 - acc: 0.5371
Epoch 3/10
18016/24999 [====================>.........] - ETA: 7:52 - loss: 0.6444 - acc: 0.5546

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))